<a href="https://colab.research.google.com/github/tuchinhung/Quantu-ML/blob/main/Comparacao_do_modelo_Classico_VS_Quantico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 

# Comment this out if you don't want to install pennylane from this notebook
!pip install pennylane
# Comment this out if you don't want to install torch from this notebook
!pip install torch
# Comment this out if you don't want to install matplotlib from this notebook
!pip install matplotlib

Modelo Classico regressao linear

In [2]:
import torch
from torch.autograd import Variable

data = torch.tensor([(0., 0.), (0.1, 0.1), (0.2, 0.2)])
print(data)

tensor([[0.0000, 0.0000],
        [0.1000, 0.1000],
        [0.2000, 0.2000]])


In [3]:
def model(phi, x=None):
  return x*phi

In [5]:
# Função de custo (a-b)^2 Perda de erro quadrático
def loss(a, b):
    return torch.abs(a - b)**2

# Perda média
def average_loss(phi):
    c = 0
    for x, y in data:
        c += loss(model(phi, x=x),  y) # Função de perda de erro quadrático
    return c

phi_new = Variable(torch.tensor(0.1), requires_grad=True)
print(phi_new)
opt = torch.optim.Adam([phi_new], lr=0.02)
print(opt)

tensor(0.1000, requires_grad=True)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.02
    weight_decay: 0
)


In [9]:
for i in range(5):
  l = average_loss(phi_new)
  print(l)
  l.backward()
  opt.step()

tensor(0.0247, grad_fn=<AddBackward0>)
tensor(0.0232, grad_fn=<AddBackward0>)
tensor(0.0218, grad_fn=<AddBackward0>)
tensor(0.0205, grad_fn=<AddBackward0>)
tensor(0.0191, grad_fn=<AddBackward0>)


Modelo Quantico Circuito

In [10]:
from pennylane import *
import torch
from torch.autograd import Variable

data1 = [(0., 0.), (0.1, 0.1), (0.2, 0.2)]
print(data1)

[(0.0, 0.0), (0.1, 0.1), (0.2, 0.2)]


In [11]:
# 'default.qubit': um simulador de estado simples de arquiteturas de circuitos quânticos baseadas em qubit.
# com 2 qubit
dev = device('default.qubit', wires=2)

@qnode(dev, interface='torch')
def circuit_model(phi, x=None):

    templates.AngleEmbedding(features=[x], wires=[0])
    templates.BasicEntanglerLayers(weights=phi, wires=[0, 1])
    return expval(PauliZ(wires=1)) # Valor de expectativa do observável fornecido.
    # o circuito a seguir retorna o valor esperado do operador Pauli Z observável no fio 1


In [12]:
# Função de custo (a-b)^2 Perda de erro quadrático
def loss(a, b):
    return torch.abs(a - b)**2

# Perda média
def average_loss(phi):
    c = 0
    for x, y in data:
        c += loss(circuit_model(phi, x=x),  y) # Função de perda de erro quadrático
    return c

phi_new = Variable(torch.tensor([[0.1, 0.2], [-0.5, 0.1]]), requires_grad=True)
print(phi_new)
opt = torch.optim.Adam([phi_new], lr=0.02)
print(opt)

tensor([[ 0.1000,  0.2000],
        [-0.5000,  0.1000]], requires_grad=True)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.02
    weight_decay: 0
)


In [13]:
for i in range(5):
  l = average_loss(phi_new)
  print(l)
  l.backward()
  opt.step()

tensor(1.6556, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1.5672, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1.4789, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1.3892, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1.2978, dtype=torch.float64, grad_fn=<AddBackward0>)
